In [ ]:
dir = "/home/jim/Playing/cf/abair-corpora/"
col_names = ["id", "set", "gender", "url", "text", "phonemes"]

In [ ]:
import os

In [ ]:
inputs = []
for (path, dirs, files) in os.walk(dir):
    if "corpusfile.txt" in files:
        inputs.append(os.path.join(path, "corpusfile.txt"))

In [ ]:
from datasets import load_dataset
dset = load_dataset("csv", data_files=inputs, column_names=col_names, delimiter="\t", split="train", index_col=False)

In [7]:
def irish_lc(text):
    def lcword(word):
        if word[0:1] in "nt" and word[1:2] in "AÁEÉIÍOÓUÚ":
            return word[0:1] + "-" + word[1:].lower()
        else:
            return word.lower()
    return " ".join(list(map(lcword, text.split(" "))))


In [8]:
assert irish_lc("bhúr nAthair") == "bhúr n-athair"